# Plot Geostrophic Winds from HRRR

In [1]:
# to make plots inline with the notebook
%matplotlib inline

In [ ]:
# to install the herbie library
!pip install herbie-data

In [30]:
# standard imports

from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import metpy.calc
import metpy.units

## Load in data from HRRR

In [ ]:
# this loads in the pressure level data from HRRR (prs, hrrr) at forecast hour 0 from 10/14/2024.
H = Herbie(
    "2024-10-14",
    model="hrrr",
    product="prs",
    fxx=0,
    save_dir="./hrrr/"
)

## Load in HRRR Variables

In [13]:
heights_500_hpa = H.xarray("HGT:500 mb")

In [ ]:
heights_500_hpa

In [28]:
heights_500_hpa = heights_500_hpa.drop('time')

## Make Plot

In [ ]:
# Let's make a basic plot of geopotential height
ax = EasyMap("50m", crs=heights_500_hpa.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax
height_contours = ax.contour(
    heights_500_hpa.longitude,
    heights_500_hpa.latitude,
    heights_500_hpa.gh,
    transform=pc,
    levels = np.arange(5200, 6000, 40)
)
ax.clabel(height_contours, height_contours.levels, inline=True, fontsize=10)


## Geostrophic Wind Calculation

In [35]:
# there are a lot of ways to calculate geostrophic wind with python! 

# as with many Python-related things, let's use the Metpy function to calculate geostrophic wind. 
# No need to reinvent the wheel.
geo_wind = metpy.calc.geostrophic_wind(heights_500_hpa.gh*metpy.units.units("m"), dx=3000*metpy.units.units("m"),
                                       dy = 3000 *metpy.units.units("m"),
                                       latitude=heights_500_hpa.latitude, 
                                       longitude=heights_500_hpa.longitude)

In [ ]:
# Let's make a basic plot of geopotential height
ax = EasyMap("50m", crs=heights_500_hpa.herbie.crs, figsize=[10, 8]).BORDERS().STATES().ax
height_contours = ax.contour(
    heights_500_hpa.longitude,
    heights_500_hpa.latitude,
    heights_500_hpa.gh,
    transform=pc,
    levels = np.arange(5200, 6000, 40)
)
ax.clabel(height_contours, height_contours.levels, inline=True, fontsize=10)

p = ax.pcolormesh(    heights_500_hpa.longitude,
    heights_500_hpa.latitude,
    metpy.calc.wind_speed(geo_wind[0], geo_wind[1]),
    transform=pc,
    vmin=0, vmax = 100
)
plt.colorbar(p)
plt.title("YOUR NAME HERE")
plt.savefig('geo_winds.png')